In [1]:
import pandas as pd
train = pd.read_csv("../toxiccomment/input/toxic_train.csv")
test=pd.read_csv("../toxiccomment/input/toxic_test.csv")
print(train.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  


In [2]:
#creat regular expression for tokenization
newp='["#$%&\'()*+-/:;<=>@[\]^_`{|}~]'

import re
from nltk.tokenize import RegexpTokenizer
toker = RegexpTokenizer('[a-z]+'+newp+'[a-z]+|[a-z]+\d+[a-z]+|[a-z]+')

train['comment_text']=train['comment_text'].map(str.lower)
test['comment_text']=test['comment_text'].map(str.lower)
train['token']=train['comment_text'].map(toker.tokenize)
test['token']=test['comment_text'].map(toker.tokenize)

print(train.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  explanation\nwhy the edits made under my usern...      0   
1  000103f0d9cfb60f  d'aww! he matches this background colour i'm s...      0   
2  000113f07ec002fd  hey man, i'm really not trying to edit war. it...      0   
3  0001b41b1c6bb37e  "\nmore\ni can't make any real suggestions on ...      0   
4  0001d958c54c6e35  you, sir, are my hero. any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  \
0             0        0       0       0              0   
1             0        0       0       0              0   
2             0        0       0       0              0   
3             0        0       0       0              0   
4             0        0       0       0              0   

                                               token  
0  [explanation, why, the, edits, made, under, my...  
1  [d'aww, he, matches, this, background, colour

In [3]:
import nltk
nltk.download('words')
nltk.download('names')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\summer\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\summer\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [4]:
#set for all words common in toxic comment
s_alltier={'suck', 'shut', 'faggot', 'motherfucker', 'dickhead', 'smash', 'cocksucker', 'fucking',
        'nigger', 'fuck', 'cock', 'bastard', 'moron', 'bitch', 'nigga', 'dumb', 'nipple', 'slut' ,'dog',
        'jew', 'piss', 'fucksex', 'boobs', 'fistfuck', 'stupid', 'balls', 'anal', 'fag', 'bush', 
         'trash', 'crap', 'idiot', 'pussy', 'mexicans', 'vagina', 'hitler', 'gay', 'bullshit', 'wanker',
         'murder', 'homosexuality', 'die', 'racist', 'basterd', 'motherfucking', 'bark', 'flithy', 
         'ass', 'cunt', 'fat', 'shit', 'freak', 'brain', 'bash', 'wtf', 'filthy', 'suffer', 'blood', 
         'bleed', 'deserve', 'ban', 'mad', 'harassement', 'abuse', 'suicide', 'sexuality', 'hack', 
           'safe', 'cut', 'rape', 'swear', 'knife', 'shoot', 'splatter', 'hook', 'sack', 'dick', 'penis', 'damn', 
         'life', 'pit', 'necrophiliac', 'legal', 'destroy', 'corpse', 'kill', 'gut', 'cancer', 'horrible', 
           'gun', 'nut', 'dirty', 'hit', 'punch', 'violence', 'stab'}


#set for english dictionary 
from nltk.corpus import words
word_list=words.words()
word_list.append('fuck')
s=set()
s.update(word_list)
s.update(s_alltier)

#set for name dictionary
from nltk.corpus import names
name_list=names.words()
for a in range(0,len(name_list),1):
    name_list[a]=name_list[a].lower()
s.update(name_list)

# specific set of words for toxic.. comment
s_test={'idiot', 'tail', 'fucking', 'dick', 'sucker', 'faggot', 'pisser', 'bastard', 'retard', 'vagina', 'queer', 'dumb', 'suck', 'freak', 'cock', 'suicide', 'fucker', 'hitler', 'murderer', 'nigger', 'wank', 'vaginal', 'homosexuality', 'ball', 'fuck', 'cocksucker', 'barky', 'motherfucker', 'nigga'}


In [5]:

from nltk.metrics import edit_distance

def dupletter_spellcor_clean(df: pd.DataFrame):
    for i,row in df.iterrows():
        print('we are at', i)  
        if i ==1000:
            break
        for i2, token in enumerate(row['token']):
            if token in s:
                continue
        # correct word to words for toxic.. comment(e.g. faggat --> faggot)
            next=True
            for word in s_test:
                if edit_distance(token,word)==1:
                    print('spell-toxic letter: ',row['token'][i2], 'is corrected as')
                    df.loc[i,'token'][i2]=word
                    next=False
                    print(word)
                    break

            if next==False:
                continue
        
        # correct word with duplicate letter(e.g. goooooooood --> good)
            a=0
            while (len(token)-1)>a>=0:
                if token[a]==token[a+1]:
                    token=token[:a]+token[(a+1):]
                    if token in s_alltier:
                        print('dupletter-toxic letter: ',row['token'][i2],'is correct as ',token)
                        df.loc[i,'token'][i2]=token
                        break
                    if token in s:
                        print('dupletter-normal letter: ',row['token'][i2],'is correct as ',token)
                        df.loc[i,'token'][i2]=token
                        break
                else:
                    a+=1
    print('end of clean of ', df)

dupletter_spellcor_clean(train)
dupletter_spellcor_clean(test) 
        
        

we are at 0
we are at 1
we are at 2
we are at 3
we are at 4
we are at 5
we are at 6
we are at 7
we are at 8
we are at 9
we are at 10
we are at 11
we are at 12
we are at 13
we are at 14
we are at 15
dupletter-normal letter:  neff is correct as  nef
dupletter-normal letter:  neff is correct as  nef
we are at 16
dupletter-normal letter:  comming is correct as  coming
we are at 17
we are at 18
we are at 19
dupletter-normal letter:  acctually is correct as  actually
we are at 20
we are at 21
we are at 22
we are at 23
we are at 24
dupletter-normal letter:  ii is correct as  i
we are at 25
we are at 26
we are at 27
we are at 28
we are at 29
we are at 30
we are at 31
we are at 32
we are at 33
dupletter-normal letter:  digg is correct as  dig
dupletter-normal letter:  diggs is correct as  digs
we are at 34
we are at 35
we are at 36
we are at 37
we are at 38
we are at 39
we are at 40
we are at 41
we are at 42
dupletter-normal letter:  ww is correct as  w
we are at 43
we are at 44
we are at 45
we

we are at 410
we are at 411
we are at 412
we are at 413
we are at 414
we are at 415
spell-toxic letter:  retards is corrected as
retard
spell-toxic letter:  fuk is corrected as
fuck
we are at 416
we are at 417
we are at 418
we are at 419
we are at 420
we are at 421
we are at 422
we are at 423
we are at 424
we are at 425
dupletter-normal letter:  hoppe is correct as  hope
dupletter-normal letter:  hoppe is correct as  hope
we are at 426
we are at 427
dupletter-normal letter:  warring is correct as  waring
dupletter-normal letter:  pp is correct as  p
we are at 428
we are at 429
we are at 430
we are at 431
we are at 432
we are at 433
we are at 434
we are at 435
we are at 436
we are at 437
we are at 438
dupletter-normal letter:  warring is correct as  waring
dupletter-normal letter:  rr is correct as  r
dupletter-normal letter:  rr is correct as  r
we are at 439
spell-toxic letter:  fucked is corrected as
fucker
we are at 440
we are at 441
we are at 442
we are at 443
we are at 444
we are 

we are at 789
we are at 790
we are at 791
we are at 792
we are at 793
we are at 794
we are at 795
we are at 796
we are at 797
we are at 798
we are at 799
we are at 800
we are at 801
we are at 802
we are at 803
we are at 804
we are at 805
we are at 806
we are at 807
we are at 808
we are at 809
we are at 810
we are at 811
we are at 812
we are at 813
we are at 814
we are at 815
we are at 816
we are at 817
we are at 818
we are at 819
we are at 820
spell-toxic letter:  murdered is corrected as
murderer
we are at 821
we are at 822
we are at 823
we are at 824
we are at 825
we are at 826
dupletter-normal letter:  blesss is correct as  bless
we are at 827
spell-toxic letter:  fucked is corrected as
fucker
we are at 828
we are at 829
we are at 830
we are at 831
we are at 832
we are at 833
we are at 834
we are at 835
dupletter-normal letter:  comming is correct as  coming
we are at 836
we are at 837
we are at 838
we are at 839
we are at 840
we are at 841
we are at 842
we are at 843
we are at 844


we are at 0
spell-toxic letter:  fuckin is corrected as
fucking
we are at 1
we are at 2
we are at 3
we are at 4
we are at 5
we are at 6
we are at 7
we are at 8
dupletter-normal letter:  magnittude is correct as  magnitude
we are at 9
we are at 10
we are at 11
dupletter-normal letter:  beckett is correct as  becket
we are at 12
we are at 13
we are at 14
we are at 15
we are at 16
we are at 17
we are at 18
we are at 19
we are at 20
we are at 21
we are at 22
we are at 23
we are at 24
we are at 25
dupletter-normal letter:  textt is correct as  text
we are at 26
we are at 27
we are at 28
we are at 29
we are at 30
we are at 31
we are at 32
we are at 33
we are at 34
we are at 35
we are at 36
we are at 37
we are at 38
dupletter-normal letter:  grimm is correct as  grim
dupletter-normal letter:  lapp is correct as  lap
spell-toxic letter:  sucks is corrected as
suck
we are at 39
we are at 40
we are at 41
we are at 42
dupletter-normal letter:  rickk is correct as  rick
we are at 43
dupletter-norm

we are at 343
we are at 344
we are at 345
we are at 346
we are at 347
we are at 348
we are at 349
spell-toxic letter:  fucker is corrected as
sucker
we are at 350
we are at 351
we are at 352
we are at 353
we are at 354
we are at 355
we are at 356
we are at 357
we are at 358
we are at 359
we are at 360
we are at 361
we are at 362
we are at 363
we are at 364
spell-toxic letter:  fuk is corrected as
fuck
we are at 365
dupletter-normal letter:  iii is correct as  i
we are at 366
we are at 367
we are at 368
we are at 369
we are at 370
we are at 371
we are at 372
we are at 373
we are at 374
we are at 375
we are at 376
we are at 377
dupletter-normal letter:  ii is correct as  i
spell-toxic letter:  murdered is corrected as
murderer
dupletter-normal letter:  wwii is correct as  wi
spell-toxic letter:  murdered is corrected as
murderer
dupletter-normal letter:  wwii is correct as  wi
spell-toxic letter:  murdered is corrected as
murderer
dupletter-normal letter:  wwii is correct as  wi
duplette

we are at 656
we are at 657
we are at 658
we are at 659
we are at 660
we are at 661
we are at 662
we are at 663
we are at 664
we are at 665
we are at 666
we are at 667
we are at 668
we are at 669
we are at 670
we are at 671
we are at 672
we are at 673
we are at 674
we are at 675
spell-toxic letter:  dicks is corrected as
dick
we are at 676
dupletter-normal letter:  rebbe is correct as  rebe
we are at 677
we are at 678
we are at 679
we are at 680
we are at 681
we are at 682
we are at 683
we are at 684
we are at 685
we are at 686
we are at 687
we are at 688
we are at 689
we are at 690
dupletter-normal letter:  longgggg is correct as  long
we are at 691
we are at 692
we are at 693
we are at 694
we are at 695
we are at 696
we are at 697
we are at 698
we are at 699
we are at 700
we are at 701
we are at 702
we are at 703
we are at 704
we are at 705
we are at 706
we are at 707
we are at 708
we are at 709
we are at 710
we are at 711
we are at 712
we are at 713
we are at 714
we are at 715
we ar

In [6]:
train.to_csv('clean_train.csv',index=False)
test.to_csv('clean_test.csv',index=False)